# Gauss-Newton for Logistic Regression

In [1]:
include("../jl/UCBAdmit_Logit.jl")
include("../jl/admissionll.jl")
include("../jl/admissiongr.jl")

Variable `Y` has the observed variable.
Variable `X` has the explanatory variables.


admitgr (generic function with 1 method)

## Add Backtracking to Gradient Descent

In [2]:
function backtrack(S,∇S₀,p,X₀,Y, X, counts; α₀ = 1.0, ρ = 0.5, c = 0.2)
  S₀ = S(X₀, Y, X, counts)
  relaxSlope = c*p'*(∇S₀'*S₀)
  condition(step) = 0.5*dot(S₀,S₀) + step*relaxSlope[1]

    α = deepcopy(α₀)
  Sₐ = S₀ #Ensures appropriate scope of Sₐ
  try
    Sₐ = S(X₀ + α*p, Y, X, counts)
  catch
  end
  funcEvals = 2
  while 0.5*dot(Sₐ,Sₐ) > condition(α)
    α = ρ*α
    try
      Sₐ = S(X₀ + α*p, Y, X, counts)
    catch
      continue
    end
    funcEvals += 1
  end

  return α, funcEvals
end

backtrack (generic function with 1 method)

In [3]:
function logitback(S,∇S, B; 
        Y=Y, X=X[:,1:7], counts=X[:,8], ϵ=1e-8, maxiter = 25)
  i = 0
  S₀ = S(B, Y, X, counts)
  ∇S₀ = ∇S(B, Y, X, counts)
  while norm(∇S₀) > ϵ && i <= maxiter
    i += 1
    #Search Direction
    ∇S₀ = ∇S(B, Y, X, counts)
    p = ∇S₀'
    #Step Length
    α, evals = backtrack(S,∇S₀,p,B,Y,X,counts)
    #Update Parameter
    B += α*p
    #Update Gradient
    S₀ = S(B, Y, X, counts)
  end

  return B, i
end

logitback (generic function with 1 method)

In [4]:
B = [1 1 1 1 1 1 1]
sol, iters = logitback(admitll, admitgr, B; 
    Y=Y, X=X[:,1:7], counts=X[:,8], ϵ=1e-8, maxiter=10000);
sol'

7×1 Adjoint{Float64,Array{Float64,2}}:
  0.5820509441989553  
  0.09986982421880013 
 -0.043397374253354215
 -1.2625973212260113  
 -1.2946058038223804  
 -1.7393049376118883  
 -3.3064787930710797  

In [5]:
iters

2845

## Gauss-Newton
Add the Hessian


In [ ]:
#H <- t(X)%*%diag(as.vector(1/(1+exp(-X%*%B))*(1-(1/(1+exp(-X%*%B)))))*counts)%*%X/sum(counts)
#H


In [12]:
B = [1 1 1 1 1 1 1]'

MethodError: MethodError: no method matching typed_hcat(::typeof(exp), ::Float64, ::Int32, ::Int32)
Closest candidates are:
  typed_hcat(!Matched::Type{T}, ::Number...) where T at abstractarray.jl:1209
  typed_hcat(!Matched::Type{T}, ::Union{Number, Adjoint{T,#s549} where #s549<:(AbstractArray{T,1} where T) where T}...) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win32\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\adjtrans.jl:165
  typed_hcat(!Matched::Type{T}, ::Union{Number, Transpose{T,#s549} where #s549<:(AbstractArray{T,1} where T) where T}...) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win32\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\adjtrans.jl:166
  ...

In [10]:
exp(-X[:,1:7]*B)

DimensionMismatch: DimensionMismatch("matrix is not square: dimensions are (24, 1)")